# Etapa 5: Adicionar um pipeline de implantação

Nas quatro etapas anteriores, você implementou um pipeline automatizado de processamento de dados e construção de modelos. Cada execução do pipeline produz uma nova versão do modelo. Este notebook implementa a etapa de implantação automatizada do modelo em nosso fluxo de trabalho de ML.

Você pode usar um [modelo de projeto SageMaker MLOps](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates.html) para provisionar um pipeline de CI/CD de implantação de modelos pronto para uso.

Este modelo automatiza a implantação de modelos no registro de modelos SageMaker em endpoints SageMaker para inferência em tempo real. Este modelo reconhece mudanças no registro de modelos. Quando uma nova versão do modelo é registrada e aprovada, ele inicia automaticamente uma implantação.


![](img/seis-etapas-5.png)

<div class="alert alert-info"> Certifique-se de que você está usando o kernel <code>Python 3</code> no JupyterLab para este notebook.</div>


In [1]:
import boto3
import sagemaker 
from time import gmtime, strftime, sleep

sagemaker.__version__

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Fetched defaults config from location: /home/sagemaker-user/.config/sagemaker/config.yaml


'2.226.1'

In [2]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERRO] VOCÊ DEVE EXECUTAR O NOTEBOOK 00-start-here")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")

Stored variables and their in-db values:
baseline_s3_url                         -> 's3://sagemaker-us-east-1-398333718380/from-idea-t
bucket_name                             -> 'sagemaker-us-east-1-398333718380'
bucket_prefix                           -> 'from-idea-to-prod/xgboost'
dataset_feature_group_name              -> 'from-idea-to-prod-23-08-19-42'
dataset_file_local_path                 -> 'data/bank-additional/bank-additional-full.csv'
domain_id                               -> 'd-wxmppmljqkj5'
evaluation_s3_url                       -> 's3://sagemaker-us-east-1-398333718380/from-idea-t
feature_store_bucket_prefix             -> 'from-idea-to-prod/feature-store'
initialized                             -> True
input_s3_url                            -> 's3://sagemaker-us-east-1-398333718380/from-idea-t
mlflow_arn                              -> 'arn:aws:sagemaker:us-east-1:398333718380:mlflow-t
mlflow_name                             -> 'mlflow-d-wxmppmljqkj5'
model_package_gr

In [3]:
sm = boto3.client("sagemaker")

## Criar um projeto MLOps

Siga o mesmo procedimento do notebook da etapa 4 para criar um projeto MLOps de implantação de modelo.

A Opção 1 é recomendada, pois não requer entrada manual e não tem dependência da IU.</br>
A Opção 2 é fornecida para demonstrar o fluxo da IU [**Criar Projeto**](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-create.html).




### Opção 1: Criar projeto programaticamente

Use `boto3` para criar um projeto MLOps por meio de uma API SageMaker.


In [4]:
sc = boto3.client("servicecatalog")

sc_provider_name = "Amazon SageMaker"
sc_product_name = "MLOps template for model deployment"

In [5]:
p_ids = [p['ProductId'] for p in sc.search_products(
    Filters={
        'FullTextSearch': [sc_product_name]
    },
)['ProductViewSummaries'] if p["Name"]==sc_product_name]

In [6]:
p_ids

['prod-5vmc7unyqksbs']

In [7]:
# Se você receber alguma exceção deste código, vá para a Opção 2 e crie um projeto na IU do Studio
if not len(p_ids):
    raise Exception("Nenhum produto Amazon SageMaker ML Ops encontrado!")
elif len(p_ids) > 1:
    raise Exception("Muitos produtos Amazon SageMaker ML Ops correspondentes encontrados!")
else:
    product_id = p_ids[0]
    print(f"ID do produto ML Ops: {product_id}")


ML Ops product id: prod-5vmc7unyqksbs


In [8]:
provisioning_artifact_id = sorted(
    [i for i in sc.list_provisioning_artifacts(
        ProductId=product_id
    )['ProvisioningArtifactDetails'] if i['Guidance']=='DEFAULT'],
    key=lambda d: d['Name'], reverse=True)[0]['Id']

In [9]:
provisioning_artifact_id

'pa-q4qe35zerlzhq'

In [10]:
project_name = f"model-deploy-{strftime('%-m-%d-%H-%M-%S', gmtime())}"

In [11]:
project_parameters = [
    {
        'Key': 'SourceModelPackageGroupName',
        'Value': model_package_group_name
    },
]

Finally, create a SageMaker project from the service catalog product template:

In [12]:
# Cria um projeto no SageMaker
r = sm.create_project(
    ProjectName=project_name,
    ProjectDescription="Model build project",
    ServiceCatalogProvisioningDetails={
        'ProductId': product_id,
        'ProvisioningArtifactId': provisioning_artifact_id,
        'ProvisioningParameters': project_parameters
    },
)

print(r)
project_id = r["ProjectId"]

{'ProjectArn': 'arn:aws:sagemaker:us-east-1:398333718380:project/model-deploy-7-23-11-46-25', 'ProjectId': 'p-ilq3pvs0je1i', 'ResponseMetadata': {'RequestId': '82b38ed5-69c7-4272-a606-729a794d4279', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '82b38ed5-69c7-4272-a606-729a794d4279', 'content-type': 'application/x-amz-json-1.1', 'content-length': '121', 'date': 'Tue, 23 Jul 2024 11:46:25 GMT'}, 'RetryAttempts': 0}}


<div class="alert alert-info"> 💡 <strong> Wait until project creation is completed by running the next cell</strong>
</div>

<svg width="800" height="125" viewBox="0 0 800 125" xmlns="http://www.w3.org/2000/svg">
    <defs>
        <linearGradient id="fadeGradient" x1="0" x2="1">
            <stop offset="0%" stop-color="#F0F0F0"/>
            <stop offset="100%" stop-color="#F0F0F0" stop-opacity="0"/>
        </linearGradient>
        <mask id="fadeMask">
            <rect x="0" y="0" width="750" height="125" fill="white"/>
            <rect x="750" y="0" width="50" height="125" fill="url(#fadeGradient)"/>
        </mask>
    </defs>
    <path d="M3,50 A50,50 0 0 1 53,3 L797,3 L797,97 L97,97 L50,115 L3,97 Z" fill="#F0F0F0" stroke="#E0E0E0" stroke-width="1" mask="url(#fadeMask)"/>
    <circle cx="50" cy="50" r="30" fill="#57c4f8" stroke="#57c4f8" stroke-width="1"/>
    <circle cx="50" cy="50" r="25" fill="#F0F0F0"/>
    <line x1="50" y1="50" x2="50" y2="30" stroke="#57c4f8" stroke-width="3" stroke-linecap="round"/>
    <line x1="50" y1="50" x2="65" y2="50" stroke="#57c4f8" stroke-width="3" stroke-linecap="round"/>
    <text x="100" y="34" font-family="Arial, sans-serif" font-size="14" fill="#333333">A próxima célula pode levar alguns minutos para ser executada. Por favor, seja paciente.</text>
</svg>

In [13]:
# A criação do projeto leva cerca de 3 a 5 minutos
while sm.describe_project(ProjectName=project_name)['ProjectStatus'] != 'CreateCompleted':
    sleep(10)
    print("Aguardando a conclusão da criação do projeto")

print(f"Criação do projeto MLOps {project_name} concluída")


Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
MLOps project model-deploy-7-23-11-46-25 creation completed


### Fim da Opção 1: Criar projeto programaticamente

Agora você provisionou um modelo de projeto em seu ambiente SageMaker. Navegue até a seção **Trabalhando com o projeto MLOps para implantação de modelo**.

---




### Option 2: Create a project in Studio UI
<div class="alert alert-info"> 💡 <strong> Skip this section if you created a project programmatically </strong>

In [15]:
print(f"The last used model package group is {model_package_group_name}")

The last used model package group is from-idea-to-prod-pipeline-model-08-08-30-16


Siga as instruções no Guia do Desenvolvedor – [Criar um Projeto MLOps usando o Amazon SageMaker Studio ou Studio Classic](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-create.html). Escolha a opção **Studio**.

Para o modelo, escolha **Model deployment**.
Em **Project details**, você precisa fornecer um nome e uma descrição de projeto opcional.

Este modelo tem um parâmetro obrigatório `SourceModelPackageGroupName`. Insira o nome do grupo de pacotes de modelos que você usou nos notebooks das etapas 3 e 4.

Opcionalmente, adicione tags, que são pares chave-valor que você pode usar para rastrear seus projetos.

Escolha **Create** e aguarde o projeto aparecer na lista Projetos.



### End of Option 2: Create a project in Studio UI
Now when you have the project created, move to the section **Trabalhando com o projeto MLOps para implantação de modelo**.

---

## Trabalhando com o projeto MLOps para implantação de modelo
O modelo provisiona um repositório CodeCommit com arquivos de configuração para especificar as etapas de implantação do modelo, modelos AWS CloudFormation para definir endpoints como infraestrutura e código inicial para testar o endpoint.

Este modelo fornece os seguintes recursos:

1. Um repositório AWS CodeCommit que contém código de modelo que implanta modelos em endpoints em ambientes de teste e produção
2. Um pipeline AWS CodePipeline que possui etapas `source`, `build`, `deploy-to-staging` e `deploy-to-production`. A etapa `source` aponta para o repositório CodeCommit, e a etapa `build` obtém o código desse repositório e gera pilhas CloudFormation para implantar. As etapas `deploy-to-staging` e `deploy-to-production` implantam as pilhas CloudFormation em seus respectivos ambientes. Há uma etapa de aprovação manual entre as etapas de build de teste e produção, para que um engenheiro de MLOps deva aprovar o modelo antes que ele seja implantado na produção.
3. Uma regra do Amazon EventBridge para iniciar uma execução do pipeline CodePipeline quando uma versão do pacote de modelo for aprovada ou rejeitada.
4. Há também uma etapa de aprovação manual após os testes unitários de espaço reservado. Você pode implementar seus próprios testes para substituir os testes de espaço reservado.

O modelo também implanta um bucket do Amazon S3 para armazenar artefatos, incluindo artefatos do CodePipeline e CodeBuild, e quaisquer artefatos gerados pelas execuções do pipeline SageMaker.

O diagrama a seguir mostra a arquitetura.

<img src="img/mlops-model-deploy.png" width="600"/>

Você não precisa implementar nenhuma alteração de configuração para o projeto. O pipeline de implantação de modelos funciona imediatamente.
Para iniciar o pipeline de implantação de modelos, você deve aprovar a versão do modelo no registro de modelos.



### Aprovar uma versão do modelo

Aprovar uma versão do modelo faz com que o projeto MLOps inicie o processo de implantação do modelo.

Na primeira etapa, o pipeline de implantação do modelo implanta a versão do modelo em um endpoint de inferência em tempo real SageMaker de teste.

Você pode aprovar a versão do modelo no Studio no Registro de Modelos ou fazer isso programaticamente no notebook. Vamos fazer isso programaticamente.




In [14]:
try:
    print(model_package_group_name)
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("Execute o notebook da etapa 03 para criar um pipeline, executar o pipeline e registrar uma versão do modelo no registro de modelos")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")


from-idea-to-prod-pipeline-model-23-07-51-02


In [15]:
# lista todos os pacotes de modelos e seleciona o mais recente
model_packages = []

for p in sm.get_paginator('list_model_packages').paginate(
        ModelPackageGroupName=model_package_group_name,
        SortBy="CreationTime",
        SortOrder="Descending",
    ):
    model_packages.extend(p["ModelPackageSummaryList"])

if len(model_packages) == 0:
    raise Exception(f"Nenhum pacote de modelo foi encontrado para o grupo de pacotes de modelos {model_package_group_name}. Execute primeiro um pipeline de criação de modelo.")

print(f"Existem {len(model_packages)} versões de modelo no grupo de pacotes de modelos {model_package_group_name}")
print(f"Aprovar o pacote de modelo mais recente:")

latest_model_package_arn = model_packages[0]["ModelPackageArn"]
print(latest_model_package_arn)


There are 5 model versions in the from-idea-to-prod-pipeline-model-23-07-51-02 model package group
Approve the most recent model package:
arn:aws:sagemaker:us-east-1:398333718380:model-package/from-idea-to-prod-pipeline-model-23-07-51-02/5


A instrução a seguir define o `ModelApprovalStatus` para o pacote de modelo mais recente no registro de modelos como `Aprovado`. A alteração de estado do pacote de modelo inicia a regra EventBridge e a regra inicia o pipeline de CI/CD CodePipeline com a implantação do modelo.


In [16]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=latest_model_package_arn,
    ModelApprovalStatus="Approved",
)

Você pode ver que a última versão do modelo no registro do modelo Studio UI alterou o **Status** para `Approved`:

![](img/model-registry-version-approved.png)

### Execução do pipeline de implantação
Após a aprovação de uma versão do modelo na célula de código acima, o pipeline de CI/CD de implantação do modelo executa as seguintes ações:
1. Criar arquivos de configuração de parâmetros do CloudFormation com parâmetros de staging e produção para os templates do CloudFormation com IaC do endpoint do SageMaker
2. Criar um endpoint de inferência em tempo real do SageMaker com o nome `<NOME-DO-PROJETO>-staging` na conta atual
3. Executar o script de teste no endpoint de staging
4. Aguardar até que o resultado do teste seja aprovado manualmente no [console do AWS CodePipeline](https://console.aws.amazon.com/codesuite/codepipeline)
5. Criar um endpoint do SageMaker com o nome `<NOME-DO-PROJETO>-prod` na conta atual

Aguarde cerca de 10-15 minutos até que o pipeline conclua a implantação do endpoint de staging. Você pode ver o status do endpoint na interface do Studio em **Implantações** > **Endpoints**:

![](img/endpoint-staging-creating.png)

Depois que o status do endpoint mudar de `Creating` para `InService`, o endpoint de staging estará totalmente operacional. Você pode iniciar o processo de implantação do modelo para o estágio de produção aprovando manualmente o estágio **DeployStaging** do pipeline do CodePipeline. Na próxima seção, você aprovará a implantação do modelo e iniciará a segunda etapa da implantação em um endpoint de produção.

Para ver os endpoints de inferência na interface do Studio, clique no link construído pela célula de código abaixo:

In [19]:
from IPython.display import HTML

# Monstre o link de do endpoint de inferência
display(
    HTML('<b>Veja <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/inference-experience/endpoints">a inferência do endpoint</a> no Studio UI</b>'.format(
            domain_id, region))
)

### Implantar a versão do modelo em produção
<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">Aguarde até que o status do endpoint de testes mude para InService, e então continue com as células de código a seguir.
    </p>
</div>

Vamos construir um link de aprovação do CodePipeline.

Se você usou a opção 1 `boto3` para criar um projeto de MLOps, o `project_name` e o `project_id` são configurados automaticamente. Você pode executar a seguinte célula de código para imprimir os valores. Se você seguiu as instruções da interface para criar um projeto, deve definir o `project_name` manualmente.

In [20]:
try:
    print(project_name)
    print(project_id)
except NameError:
    print("++++++++++++++++++++++++++++++++++++++")
    print("Você deve definir o project_name manualmente")
    print("++++++++++++++++++++++++++++++++++++++")

model-deploy-7-23-11-46-25
p-ilq3pvs0je1i


In [21]:
# Defina o nome do projeto de implantação do modelo se você não tiver usado a implantação baseada em boto3
# Project_name = “<USE YOUR PROJECT NAME>”


# Obter o ID do projeto
project_id = sm.describe_project(ProjectName=project_name)['ProjectId']

# Construir o nome do pipeline do CodePipline
code_pipeline_name = f"sagemaker-{project_name}-{project_id}-modeldeploy"

In [22]:
# Mostre o link de aprovação Show the approval link
display(
    HTML(
        '<b>Por favor, aprove a etapa manual em <a target="top" href="https://console.aws.amazon.com/codesuite/codepipeline/pipelines/{}/view?region={}">AWS CodePipeline</a></b>'.format(
            code_pipeline_name, region)
    )
)

Clique no link ^^^ acima ^^^ para abrir o console do CodePipeline com o fluxo de trabalho de execução do pipeline.

In the **DeployStaging stage**, choose **Review** on the **ApproveDeployment** step. Note, you might wait until `TestStaging` step completes with `Succeeded` status. 

![](img/deploy-staging-review.png)

In the **Review** dialog box, select **Approve** and choose **Submit**:

![](img/approve-deployment.png)

Approving the **DeployStaging** stage causes the deployment pipeline to continue and to deploy the model to the production endpoint. To view the endpoints, choose the **Deployments** > **Endpoints** in Studio UI.

In [23]:
# Mostrar o link do endpoint da inferência
display(
    HTML('<b>Veja <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/inference-experience/endpoints">a inferencia do endpoint</a> no  Studio UI</b>'.format(
            domain_id, region))
)

Conforme seu pipeline de implantação CI/CD continua, você verá o endpoint de produção no status `Creating` junto com o endpoint de staging previamente implantado no status `InService`:

![](img/endpoint-prod-creating.png)

Após 10-15 minutos, a implantação é concluída e ambos os endpoints estão no status `InService`.

Navegue até o Studio e escolha **Deployments** > **Projects**. No painel Project, selecione o projeto `model-deploy-<TIMESTAMP>`. No painel de detalhes do projeto, selecione **Endpoints**. Você verá que ambos os endpoints, `staging` e `prod`, estão visíveis no projeto de implantação porque o projeto e os endpoints estão conectados via metadados:

![](img/project-endpoints.png)

## Resumo

Neste notebook, você implementou um pipeline de implantação CI/CD automatizado com as seguintes características:
- uso de templates IaC (Infraestrutura como Código) do CloudFormation para implantação de endpoints de inferência em tempo real do SageMaker
- aprovação do modelo no registro de modelos inicia o pipeline de implantação do modelo
- o pipeline de implantação do modelo contém dois estágios, staging e produção, com testes automatizados para o endpoint de staging e aprovação manual para a implantação em produção

---

## Limpeza
<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">
    Se você vai executar o notebook da etapa 6 (Monitoramento de Qualidade de Dados e Modelos), você precisa manter pelo menos um dos endpoints. Se você terminar o treinamento aqui e não executar o notebook da etapa 6, navegue até o <b>notebook de limpeza (99-clean-up.ipynb)</b> e siga as instruções de limpeza para evitar cobranças em sua conta AWS.
    <br>
    <br>
    Você não precisa executar a limpeza se estiver usando uma conta AWS fornecida pela AWS.
    </p>
</div>

## Continue com a etapa 6
Abra o [notebook](06-monitoring.ipynb) da etapa 6.

## Ideias de desenvolvimento adicional para seus projetos do mundo real
- Adicionar criptografia de ponta a ponta dos dados usando chaves AWS KMS
- Criar um [modelo personalizado de projeto SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-custom.html) para implantação de modelos que atenda aos requisitos específicos do seu projeto
- Adicionar [implantação de modelos em múltiplas contas](https://aws.amazon.com/blogs/machine-learning/multi-account-model-deployment-with-amazon-sagemaker-pipelines/) ao seu fluxo de trabalho de ML
- Adicionar testes automatizados de modelo ao espaço reservado no pipeline do CodePipeline
- Usar o [Amazon SageMaker Inference Recommender](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-recommender.html) para executar testes de carga automatizados para seus endpoints de inferência e selecionar o melhor tipo de instância e configuração

## Recursos adicionais
- [Implantar um modelo de Machine Learning em um endpoint de inferência em tempo real](https://aws.amazon.com/getting-started/hands-on/machine-learning-tutorial-deploy-model-to-real-time-inference-endpoint/)
- [Walkthrough do projeto SageMaker MLOps](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough.html)
- [Laboratório do Amazon SageMaker Pipelines no SageMaker Immersion Day](https://catalog.us-east-1.prod.workshops.aws/workshops/63069e26-921c-4ce1-9cc7-dd882ff62575/en-US/lab6)
- [Secure MLOps no Amazon SageMaker](https://github.com/aws-samples/amazon-sagemaker-secure-mlops)
- [Abordagens de teste para modelos ML no Amazon SageMaker](https://aws.amazon.com/blogs/machine-learning/testing-approaches-for-amazon-sagemaker-ml-models/)
- [Série de blogs sobre padrões de hospedagem de modelos no Amazon SageMaker](https://aws.amazon.com/blogs/machine-learning/model-hosting-patterns-in-amazon-sagemaker-part-1-common-design-patterns-for-building-ml-applications-on-amazon-sagemaker/)
- [Aproveite as estratégias de implantação avançadas usando as guardrails de implantação do Amazon SageMaker](https://aws.amazon.com/blogs/machine-learning/take-advantage-of-advanced-deployment-strategies-using-amazon-sagemaker-deployment-guardrails/)
- [Melhores práticas de implantação de MLOps para endpoints de serviço de modelo de inferência em tempo real com o Amazon SageMaker](https://aws.amazon.com/blogs/machine-learning/mlops-deployment-best-practices-for-real-time-inference-model-serving-endpoints-with-amazon-sagemaker/)

# Desligar o kernel

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>